In [12]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [13]:
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    return Browser("chrome", **executable_path, headless=False)

In [14]:
def scrape():
    browser = init_browser()
    nasa_data = {}
    url1 = 'https://mars.nasa.gov/news/'
    url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    
    browser.visit(url1)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    #collect the latest news title and paragraph text (latest result will be first in list; index 0 of the scaping results)
    nasa_data["title"] = soup.find_all("div", class_="list_text")[0].find("a").text
    nasa_data["paragraph"] = soup.find_all("div", class_="list_text")[0].find("div", class_="article_teaser_body").text
    
    browser.visit(url2)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    #scrape image URL from website
    relative_image_path = soup.find_all("img", class_="headerimage fade-in")[0]["src"]
    #concatenate relative image path to website URL
    nasa_data["featured_image_url"] = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + relative_image_path
    
    browser.quit()
    
    return nasa_data

In [15]:
scrape()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Garrett_Personal\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


{'title': "NASA's Perseverance Pays Off Back Home",
 'paragraph': 'Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'}

In [16]:
#using pandas to scrape tabular data from website
url3 = 'https://space-facts.com/mars/'
tables = pd.read_html(url3)
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
html_table = df.to_html() #converting the data frame into an HTML string
html_table = html_table.replace('\n', '') #stripping the new line characters from the HTML
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </

In [18]:
mars_hemisphere_data = {} #dictionary to store the image URL and the title to describe which hemisphere we are seeing
hemisphere_image_urls = [] #list to store the dictionary of Mars hemisphere data

url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = init_browser() #this is duplicate code and will need to be removed when everything goes together in the scrape function
browser.visit(url4)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

subpages = soup.find_all('a', class_='itemLink product-item')

for page in subpages:# scraping the names of the four Martian hemispheres from the site
    if page.text != "": #accounting for null values          
        browser.links.find_by_partial_text(page.text).click() #look for the title of the page and click on the link
        soup = BeautifulSoup(browser.html, "html.parser") #updating the BeautifulSoup Browser HTML to the page we are currently on
        mars_hemisphere_data["title"] = page.text #adding the title of the page to the dictionary
        mars_hemisphere_data["img_url"] = "https://astrogeology.usgs.gov" + soup.find_all('img', class_='wide-image')[0]["src"]
        hemisphere_image_urls.append(mars_hemisphere_data.copy()) #copy is necessary since dict is defined outside loop
        browser.back() #go to the previous webpage
print(hemisphere_image_urls)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Garrett_Personal\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
